# Нестандартное использование тестовой выборки

In [49]:
import scipy
import numpy as np
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Стандартная подготовка данных, но **у** мержится со "своей" неделей

In [42]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
ind = np.arange(2,36)

In [43]:
def calc_date(x):
    x[1] = x[1] - x[2];
    if (x[1] > 0):
        return x
    
    x[0] -= 1
    x[1] += 52
    return x

In [44]:
X = train.copy().take(ind, axis=1)
X[['year', 'week', 'shift']] = X[['year', 'week', 'shift']].apply(lambda x : calc_date(x), axis=1)
X = X[np.invert(X.duplicated(['item_id', 'year', 'week']))]
X = X.drop('shift', axis=1)

In [45]:
y = train.get(['y', 'week', 'year', 'item_id'])
y = y[np.invert(y.duplicated(['item_id', 'year', 'week']))]

In [46]:
df = pd.merge(X, y, on=['item_id', 'year', 'week'])

Посмотрим на f30 и y для одной недели

In [47]:
df = df.get(['f30', 'y'])

In [51]:
df.corr()

,f30,y
f30,1.0,1.0
y,1.0,1.0


Полная зависимость. Далее я хотел честно что-нибудь обучить, но 1 минута с калькулятором в руках и train-выборокой дали эту формулу:

In [52]:
df['Y_pred']=df['f30'].apply(lambda x: math.floor(x*1.61))
df['delta'] = df['Y_pred'] - df['y']
df[df.delta!=0]

,f30,y,Y_pred,delta
23557,0.0,48,0.0,-48.0
23558,0.0,80,0.0,-80.0
23559,0.0,563,0.0,-563.0


Полное совпадение, не считая трех выбросов с нулевыми признаками

Теперь можно вынимать ответы из тестовой выборки:

In [30]:
X = test.copy()
X[['year', 'week', 'shift']] = X[['year', 'week', 'shift']].apply(lambda x : calc_date(x), axis=1)
X = X[np.invert(X.duplicated(['item_id', 'year', 'week']))]
X = X.get(['year', 'week', 'item_id', 'f30'])

In [31]:
y = test.copy().get(['Num', 'week', 'year', 'item_id'])
df = pd.merge(X, y, on=['item_id', 'year', 'week'])

In [32]:
X[X.item_id==20447918]

,year,week,item_id,f30
0,2014,52,20447918,1290.0
226,2015,1,20447918,422.0
678,2015,2,20447918,1690.0
1364,2015,3,20447918,930.0
1799,2015,4,20447918,1100.0


In [33]:
df

,year,week,item_id,f30,Num
0,2015,3,20447902,16791.0,348623
1,2015,3,20447902,16791.0,349520
2,2015,3,20447902,16791.0,350192
3,2015,3,20447732,184820.0,348624
4,2015,3,20447732,184820.0,349521
5,2015,3,20447732,184820.0,350193
6,2015,3,20443951,18305.0,348625
7,2015,3,20443951,18305.0,349522
8,2015,3,20443951,18305.0,350194
9,2015,3,20443944,10.0,348626


In [34]:
df['l']=df['f30'].apply(lambda x: math.floor(x*1.61))
df = df.sort_values('Num', ascending=True)

In [36]:
ans = df.get(['Num', 'l'])

In [37]:
ans

,Num,l
42,348622,1497.0
0,348623,27033.0
3,348624,297560.0
6,348625,29471.0
9,348626,16.0
12,348627,146242.0
15,348628,67735.0
18,348629,111941.0
21,348630,102631.0
24,348631,3542.0


In [38]:
ans.to_csv("leak.tsv", sep=',', index=False)